In [1]:
import os
import sys
from typing import Optional, Any

from dotenv import load_dotenv, find_dotenv
import numpy as np

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage


# new import
from langchain_google_genai import GoogleGenerativeAIEmbeddings # embedding from Google
from langchain_community.vectorstores import Chroma # access chroma from LangChain

/Users/aditya/anaconda3/envs/mini/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# make sure GOOGLE_API_KEY (or any other llm provider, must exactly the same)
_ = load_dotenv(find_dotenv())
# or put it into variable and pass it to llm model 
# google_api_key = os.getenv('GOOGLE_API_KEY') # pass this to the model creation

# RAG Concept

![Rag Flow](../media/rag_flow.png)

One of the shortcomings of LLMs is their lack of domain-specific knowledge. 
They are trained on large, publicly available datasets, but to be useful in a business setting, they need access to internal data. 
*Retrieval-Augmented Generation* (RAG) is a technique that enhances an LLM’s responses by providing additional context from relevant documents.

At a high level, RAG works by adding contextual information into the prompt. 
When a query is received, the system searches in the database for relevant documents and inserts the retrieved content into the prompt before passing it to the LLM.

An important component of RAG is the vector database, which is used to store internal data. 
This database enables us to retrieve the most relevant sentences as context for our prompts.

# Embedding
At the core of every LLM is embedding, a process that converts a string into a vector representation. 
This allows us to compare the similarity between two sentences based on their vector distance. 
The goal is to compare sentences based on their meaning rather than just their words.

In [3]:
EMBEDDING_MODEL = GoogleGenerativeAIEmbeddings(model='models/text-embedding-004')

In [4]:
vec_query = EMBEDDING_MODEL.embed_query("Hello World")

In [5]:
vec_query = np.array(vec_query)

In [6]:
vec_query

array([ 1.41921565e-02, -9.10687726e-03, -4.07615900e-02,  3.78295407e-03,
       -5.71815018e-03, -5.98200085e-03,  5.99906370e-02,  2.03863699e-02,
        2.55044959e-02,  4.73048165e-02, -4.99246120e-02, -1.69505260e-03,
        3.48231532e-02, -1.07618682e-02, -2.28640344e-02, -1.78383943e-02,
       -8.82386509e-03,  1.31277665e-02, -1.09334208e-01,  1.41559495e-02,
        6.40330696e-03,  7.62484781e-03, -2.09325720e-02, -4.91950065e-02,
       -1.12417219e-02, -4.25484381e-04,  1.31445769e-02,  2.33667046e-02,
        1.38423229e-02,  4.45795283e-02, -4.18479554e-02,  5.61341196e-02,
       -2.10907334e-03, -2.36702487e-02,  6.81589311e-03,  6.86355727e-03,
       -5.09277508e-02,  2.82000564e-02,  7.36088492e-03, -5.37831187e-02,
       -2.64719054e-02,  3.88621613e-02,  7.20694754e-03,  1.37184178e-02,
        5.10124974e-02, -2.18461026e-02,  2.15280335e-02, -2.06419057e-03,
        1.32721965e-03,  5.67283598e-04,  4.12434302e-02,  1.10045355e-02,
       -9.87707302e-02,  

In [7]:
vec_query.shape # Google Text Embedding 004 embedding size is 768, different model have different size

(768,)

In [8]:
contents = [
    "Cuacanya sangat dingin hari ini",
    "Buku terbaru yang ada di toko sangat bagus",
    "Taman itu berada di daerah Jakarta Kota"
]
vec_contents = np.array([EMBEDDING_MODEL.embed_query(_) for _ in contents])

In [9]:
vec_contents.shape

(3, 768)

In [10]:
query = "Di dekat sini terdapat area hijau yang bagus dan rimbun"
vec_query = np.array(EMBEDDING_MODEL.embed_query(query))

In [11]:
# https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy
# euclidean distance, the lower means more similar
np.linalg.norm(vec_query-vec_contents, axis=1)

array([0.96018157, 0.95823276, 0.78226486])

# Vector Database

Vector Database is a database that are specifically tailord to store and query vector data. 
Vector database natively support similarity search based on a vector as well, with several selection method like taking the top-k.

In [12]:
VECTORDB_DIRECTORY = '../data/chromadb/'
VECTORDB = Chroma(persist_directory=VECTORDB_DIRECTORY, embedding_function=EMBEDDING_MODEL)

/var/folders/2r/q5d4kfwn6j5fkmb2nky49gz80000gn/T/ipykernel_48919/1554868473.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  VECTORDB = Chroma(persist_directory=VECTORDB_DIRECTORY, embedding_function=EMBEDDING_MODEL)


In [13]:
# save content into database
# before data could be inserted, we need to convert it into Document object
from langchain_core.documents import Document
lc_documents = []
for c_idx, c in enumerate(contents):
    lc_documents.append(Document(c, metadata={'content_id': c_idx}))

In [14]:
lc_documents

[Document(metadata={'content_id': 0}, page_content='Cuacanya sangat dingin hari ini'),
 Document(metadata={'content_id': 1}, page_content='Buku terbaru yang ada di toko sangat bagus'),
 Document(metadata={'content_id': 2}, page_content='Taman itu berada di daerah Jakarta Kota')]

In [15]:
VECTORDB.add_documents(lc_documents)

['adfe9cab-9b55-48e5-9926-b2a7a2812ec3',
 'a3951782-c1ae-479e-a22e-5ac36d585361',
 '2e66966e-86f4-46a7-a632-42a7f74bb725']

In [16]:
# retrieve data based on query
retriever = VECTORDB.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 1, "score_threshold": 0.1})
retriever.invoke(query)

[Document(metadata={'content_id': 2}, page_content='Taman itu berada di daerah Jakarta Kota')]

# RAG

For demonstration purposes, we will crawl several news data with different topic. 

In [17]:
# data loading
import json
from pprint import pprint

with open('../data/rag/data.json', 'r') as f_:
    data = json.load(f_)

pprint(data[0])

{'content': 'KOMPAS.com - Shin Tae-yong telah diberhentikan dari tugas melatih '
            'timnas Indonesia. Kendati demikian, STY tetap ingin memajukan '
            'sepak bola Indonesia melalui akademinya\n'
            '\n'
            'Lembar perjalanan Shin Tae-yong sebagai pelatih timnas Indonesia '
            'resmi berakhir.\n'
            '\n'
            'Ketua Umum PSSI, Erick Thohir, menyampaikan langsung keputusan '
            'pemberhentian STY dalam sebuah sesi konferensi pers di Menara '
            'Danareksa, Jakarta, Senin (6/1/2025)\n'
            '\n'
            'Shin Tae-yong pun mesti mengakhiri pengabdian bersama timnas '
            'Indonesia lebih cepat, meski sejatinya ia masih terikat kontrak '
            'sampai 2027.\n'
            '\n'
            'Baca juga: Shin Tae-yong, Timnas Indonesia, dan Hierarki di '
            'Budaya Korea Selatan\n'
            '\n'
            'Kepergian STY dari kursi pelatih timnas Indonesia sempat '
            '

In [ ]:
lc_documents = []
for d_idx, d in enumerate(data):
    lc_documents.append(Document(d['content'], metadata={'title': d['title'], 'url': d['url']}))

In [21]:
print(lc_documents[0].page_content)

KOMPAS.com - Shin Tae-yong telah diberhentikan dari tugas melatih timnas Indonesia. Kendati demikian, STY tetap ingin memajukan sepak bola Indonesia melalui akademinya

Lembar perjalanan Shin Tae-yong sebagai pelatih timnas Indonesia resmi berakhir.

Ketua Umum PSSI, Erick Thohir, menyampaikan langsung keputusan pemberhentian STY dalam sebuah sesi konferensi pers di Menara Danareksa, Jakarta, Senin (6/1/2025)

Shin Tae-yong pun mesti mengakhiri pengabdian bersama timnas Indonesia lebih cepat, meski sejatinya ia masih terikat kontrak sampai 2027.

Baca juga: Shin Tae-yong, Timnas Indonesia, dan Hierarki di Budaya Korea Selatan

Kepergian STY dari kursi pelatih timnas Indonesia sempat menimbulkan pertanyaan tentang nasib akademi sang pelatih asal Korea Selatan di Tanah Air.

Seperti diketahui, pada November 2024 silam, Shin TaeYong Football Academy, mulai beroperasi di Indonesia.

View this post on Instagram A post shared by Shin TaeYong Football Academy (@shintaeyong_academy)

Shin TaeY

In [22]:
# rebuild database
try:
    VECTORDB.delete_collection()
except:
    # pass if collection does not exist
    pass
# the Vector Database class requires an embedding model to automatically convert string to embedding
VECTORDB = Chroma(persist_directory=VECTORDB_DIRECTORY, embedding_function=EMBEDDING_MODEL)
# without chunking
VECTORDB.add_documents(lc_documents)

['e77d7b8d-48a5-4602-937e-2e0d2a3d7a3d',
 '4d44a7ff-d450-4446-bff4-c3e28d78e181',
 'a6df8745-8836-4ce0-b432-dcee952af14e',
 '9efe6493-4806-4861-bcd6-46ac08a34139',
 '4e7f8f3b-ecf3-4f69-8820-5145da89383d',
 'e6343fa3-47a0-49c7-9fcc-12b5370f2217',
 'c4caefa1-eb83-4649-954b-8852f33a5c24',
 '31c7112b-a5f7-49a8-a7e5-7542ea9ca599',
 'e0d3ebd7-f91c-4280-9b91-5d09af7cdb09',
 'e19288af-c61e-44da-bdd8-20dca9c9aa15',
 'c20cdffe-7f8a-4735-b5c0-2d4af6b75bf4',
 'dcc7fef5-743a-4db9-a61a-15bb0a31194c',
 'f4d6e11c-9af9-402b-ad60-1444fcaa142b',
 '3b31358f-1ad4-4f61-92af-319db133c0e2',
 '67492148-89f0-4169-9991-c5a00db745b3',
 '5da51dad-0890-4d8a-93de-063fe19bdf0f',
 '722cab26-c938-42e5-9582-8be779ebb375',
 'c5007892-ebb3-408b-bd72-91810fe364cf',
 '4125900f-b922-4510-8817-2d1a6fd8340c',
 '7ccb8047-59c1-4432-9287-6b4bfe39d7bf',
 '30710958-d6f7-4fb2-80d2-83d0dcc2441e']

In [25]:
# Chunking document: splitting the document into smaller pieces. 
# Recommended for LLMs that have a maximum token capacity that is not too large.

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(lc_documents)

In [26]:
print(docs[0].page_content)

KOMPAS.com - Shin Tae-yong telah diberhentikan dari tugas melatih timnas Indonesia. Kendati demikian, STY tetap ingin memajukan sepak bola Indonesia melalui akademinya

Lembar perjalanan Shin Tae-yong sebagai pelatih timnas Indonesia resmi berakhir.

Ketua Umum PSSI, Erick Thohir, menyampaikan langsung keputusan pemberhentian STY dalam sebuah sesi konferensi pers di Menara Danareksa, Jakarta, Senin (6/1/2025)

Shin Tae-yong pun mesti mengakhiri pengabdian bersama timnas Indonesia lebih cepat, meski sejatinya ia masih terikat kontrak sampai 2027.

Baca juga: Shin Tae-yong, Timnas Indonesia, dan Hierarki di Budaya Korea Selatan

Kepergian STY dari kursi pelatih timnas Indonesia sempat menimbulkan pertanyaan tentang nasib akademi sang pelatih asal Korea Selatan di Tanah Air.

Seperti diketahui, pada November 2024 silam, Shin TaeYong Football Academy, mulai beroperasi di Indonesia.

View this post on Instagram A post shared by Shin TaeYong Football Academy (@shintaeyong_academy)


In [27]:
# rebuild database
try:
    VECTORDB.delete_collection()
except:
    # pass if collection does not exist
    pass
# the Vector Database class requires an embedding model to automatically convert string to embedding
VECTORDB = Chroma(persist_directory=VECTORDB_DIRECTORY, embedding_function=EMBEDDING_MODEL)
# with chunking
VECTORDB.add_documents(docs)

['88979c75-224c-4853-875c-914ef1da954c',
 '5562ac25-8cb2-43c1-9821-47d7fc7bdbf8',
 '02e51e1c-4d3b-445f-a01b-8355f1a880ee',
 '5cb29158-7913-489d-a7aa-38a91ed0ff2a',
 'dc208255-2f10-4489-b0d8-be51c70395ec',
 '4d3eec09-223f-4325-82ad-e77787fbad21',
 'e623fb70-a2e5-402e-9ecf-5923e9dcec34',
 '9b46f0e9-1443-4afe-b66d-77cdd4c3be35',
 '9983594e-e3f4-42da-a30b-9095cd67a7d5',
 '0b54285d-996a-4226-863c-324224c75572',
 '95dd0d7e-8bfa-4b88-903c-893a7a5a1aca',
 '46f42e14-7bd6-429a-b830-43b4e10d6ca1',
 'd3df6afc-a3b8-41e2-a85d-b946af4508bb',
 '035f6738-a335-4400-92a9-75e24c570d3d',
 '3a85608e-55d8-4964-94fa-bfd89e88eb3c',
 '8e54efd2-d94c-445c-88cc-19a2716fedee',
 'ebfd2cfb-f93b-462e-98f1-3e7cc275f77c',
 '5040343a-49ff-4ad6-af64-cf765bcabe64',
 'ff1a7dad-8555-4b5d-be71-14e031f6d9a8',
 '22890bb5-9173-467e-a17d-a60c6f095ba6',
 '4b8fdaf0-a684-460f-8c17-bd6f1fa42ae1',
 '11db1494-d1f7-4f44-96b4-b36e9cc98eb9',
 '62b44868-94d9-4d8b-bc3c-a8bbd5e95106',
 '029f6a65-01b6-4055-8bba-4a7b30e814b0',
 '127e59c5-7af3-

In [57]:
# Given a string, we calculate the top k similar document 
def find_similar_documents(q: str, k: Optional[int]=1) -> list[str]:
    return map(
        lambda _: _.page_content, # get only page_content from result
        VECTORDB.as_retriever(
            search_type="similarity_score_threshold", 
            search_kwargs={"k": k, "score_threshold": 0.1}
        ).invoke(q)
    )

In [58]:
def combine_query_with_documents(q: str, docs: list[str]) -> str:
    context_str = "\n".join(docs)
    final_q = f"""
Konteks tambahan yang digunakan adalah berikut.
---------------------
{context_str}
---------------------
Dengan mempertimbangkan hanya informasi konteks dan bukan pengetahuan sebelumnya, \
jawab pertanyaan dibawah dengan benar dan tolak jika jawabannya tidak ditemukan \
pada konteks di atas.
Pertanyaan: {q}
    """
    return final_q

In [59]:
q = 'Kenapa STY diganti?'
similar_docs = find_similar_documents(q)
q_with_context = combine_query_with_documents(q, similar_docs)

print(q_with_context)


Konteks tambahan yang digunakan adalah berikut.
---------------------
Baca juga: Marc Klok Ungkap Problem STY di Timnas Indonesia, Hierarki dan Komunikasi

"Banyak orang Belanda yang bergabung dan bagi mereka hierarki itu sedikit berbeda.”

“Federasi, mereka menyimpulkan bahwa tim harus menjadi lebih utuh, dengan sedikit penekanan pada hierarki dan lebih banyak bekerja sama dan menjadi satu kesatuan,” ucap Klok soal alasan PSSI mendepak STY."

Hierarki di budaya Korea Selatan ini juga menjadi topik diskusi Kompas.com dengan jurnalis Korea Selatan, Jason Lee, dalam wawancara pada Rabu (8/1/2025).

"Dalam suatu level, hierarki ini ada tetapi juga sangat tergantung dari manajer itu sendiri," tuturnya melalui sambungan telekonferensi.

"Beberapa manajer lebih terbuka terhadap masukan dan diskusi taktik. Tapi juga tergantung ke kualitas pemain. Misalnya, Shin Tae-yong masih pelatih Korsel dan Son Heung-min datang berbicara untuk bahas taktik, apakah ia akan bereaksi buruk? Menurut saya tid

In [60]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
response = llm.invoke(q_with_context)

In [61]:
response.content

'Berdasarkan konteks yang diberikan, PSSI mendepak STY karena federasi menyimpulkan bahwa tim harus menjadi lebih utuh, dengan sedikit penekanan pada hierarki dan lebih banyak bekerja sama dan menjadi satu kesatuan.  Marc Klok menghubungkan hal ini dengan perbedaan hierarki, khususnya  dengan banyaknya pemain Belanda yang bergabung dalam tim.'

In [63]:
q = 'Kapan Makan Gratis Dimulai?'
similar_docs = find_similar_documents(q, 3)
q_with_context = combine_query_with_documents(q, similar_docs)
response = llm.invoke(q_with_context)
print(response.content)

Program Makan Bergizi Gratis (MBG) resmi diluncurkan pada Senin, 6 Januari 2025.


# RAG with LCEL

LangChain provide us with another easier way to combine multiple invocation into one chain using LangChain Expression Language.
Since our vector DB can be used as a runnable (with `as_retriever` function), we could:

In [64]:
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.prompts import PromptTemplate

prompt_str = """
Konteks tambahan yang digunakan adalah berikut.
---------------------
{context}
---------------------
Dengan mempertimbangkan hanya informasi konteks dan bukan pengetahuan sebelumnya, \
jawab pertanyaan dibawah dengan benar dan tolak jika jawabannya tidak ditemukan \
pada konteks di atas.
Pertanyaan: {question}
"""

k=3
retriever = VECTORDB.as_retriever(
                search_type="similarity_score_threshold", 
                search_kwargs={"k": k, "score_threshold": 0.1}
            )
prompt = PromptTemplate.from_template(prompt_str)

RAG = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | llm
)

In [65]:
res = RAG.invoke('Kapan makan gratis dimulai?')

In [67]:
res.content

'Berdasarkan konteks yang diberikan, program makan bergizi gratis (sebelumnya disebut makan siang gratis) dimulai pada 6 Januari 2025.'